# 04. Creación de grafo de las estaciones del sistema metro de CDMX

Carga de librerías

In [1]:
import json
from datetime import datetime, timedelta

import networkx as nx
import pandas as pd

import plotly.graph_objects as go
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

from random import shuffle, sample
from math import exp

Lectura de archivo json con los tiempos de traslado entre estaciones consecutivas

In [2]:
json_file = "../output_metro/travel_times_metro.json"
with open(json_file) as input_json:
    dict_times_metro = json.load(input_json)

Construimos un par de diccionarios:
- ``travel_times_metro``: contendra información sobre las estaciones, su única conexión consecutiva y el tiempo de traslado entre el par de estaciones.
- ``location_stations``: contendrá la localización geográfica de cada estación

In [3]:
travel_times_metro = dict()
location_stations = dict()

for route_id in dict_times_metro.keys():
    stations_data = dict_times_metro[route_id]
    converted_route_times = list(map(lambda x: timedelta(hours=x.hour, minutes=x.minute, seconds=x.second), [datetime.strptime(stat_time[1][4:], "%HH%MM%SS") for stat_time in stations_data]))
    
    travel_times_metro[route_id] = [(stations_data[n][0], converted_diff_time.total_seconds()/3600.0) for n, converted_diff_time in enumerate(converted_route_times)]

    for station_data in stations_data:
        location_stations[station_data[0]] = tuple(station_data[2:])

Inciamos la construcción y definición del grafo no dirigido de todas las estaciones del sistema de metro de la CDMX

In [4]:
# Inicializamos el diccionario
metro_network = dict()

# Iniciamos la insertacion de nodos por cada ruta
route_ids = travel_times_metro.keys()
for route_id in route_ids:

    # Capturamos las estaciones por ruta y sus tiempos de traslado a su estacion consecutiva
    stations, travel_times = zip(*travel_times_metro[route_id])
    NumStations = len(stations)
    for i in range(NumStations):

        # Si no existe la estacion en el grafo, se añade
        if stations[i] not in metro_network.keys():
            if i != NumStations - 1:
                metro_network[stations[i]] = [(stations[i+1], travel_times[i+1])]
            else:
                metro_network[stations[i]] = [(stations[i-1], travel_times[i-1])]
        # Sí si existe, añadimos los elementos correspondientes a dada estacion
        else:
            if i != NumStations - 1:
                metro_network[stations[i]].append((stations[i+1], travel_times[i+1]))
            else:
                metro_network[stations[i]].append((stations[i-1], travel_times[i-1]))

# El grafo aún no contempla que, si A conecta con B entonces B conecta con A
# Iniciamos la busqueda sobre todos los nodos
for node, connections in list(metro_network.items()):
    # Exploramos cada nodo objetivo dada las conexiones actuales
    for target, time in connections:

        # Si el objetivo no está en las llaves del grafo, se añade
        if target not in metro_network:
            metro_network[target] = []

        # Caso contrario, se añade un nodo restante
        nodes_list, _ = zip(*metro_network[target])
        if node not in nodes_list:
            metro_network[target].append((node, time))

Generamos un objeto ```Graph``` para 

In [5]:
metro_graph = nx.Graph()

for node, edges in metro_network.items():
    for edge in edges:
        target, weight = edge
        metro_graph.add_edge(node, target, weight=weight)

In [6]:
node_degrees = dict(metro_graph.degree())

max_degree = max(node_degrees.values())
min_degree = min(node_degrees.values())

node_colors = [node_degrees[node] for node in metro_graph.nodes()]

edge_x = []
edge_y = []
for edge in metro_graph.edges():
    x0, y0 = location_stations[edge[0]]
    x1, y1 = location_stations[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

node_x = []
node_y = []
node_text = []
for node in metro_graph.nodes():
    x, y = location_stations[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"{node}\n# de estaciones: {node_degrees[node]}")

In [7]:
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color='gray'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    textposition="top center",
    marker=dict(
        showscale=True,
        colorscale='turbo',
        cmin=min_degree,
        cmax=max_degree,
        color=node_colors,
        size=10,
        line = dict(
            color = "black",
            width = 1
        ),
        colorbar=dict(
            thickness=15,
            title='# de estaciones conectadas',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Visualización de la red de metro de la CDMX',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper")],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)))
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

fig.show()

Se realiza una implementación de la metaheurística __Recocido Simulado__ para hallar soluciones sobre el grafo completo en base a su matriz de distancias; utilizando el algoritmo de Dijkstra para hallar la ruta más corta entre nodos no vecinos.

In [8]:
class TSP_SimulatedAnnealing:
    def __init__(self, graph:nx.Graph, nodes_to_visit: list[str], init_temp: float, min_temp: float, cool_rate: float, max_iters: int):
        """
        Initializes the simulated annealing algorithm for solving the Traveling Salesman Problem (TSP).

        Parameters:
        graph (nx.Graph): A Graph containing nodes, edges and weights for each edge
        distance_matrix (pd.DataFrame): A DataFrame containing the distances between nodes.
        nodes_to_visit (list[str]): A list of nodes to be visited.
        init_temp (float): Initial temperature for the annealing process.
        min_temp (float): Minimum temperature for the annealing process.
        cool_rate (float): Cooling rate for the temperature.
        max_iters (int): Maximum number of iterations to perform.
        """
        self.__graph = graph
        self.__dist_matrix = self.__get_distances_matrix()
        self.__nodes_to_visit = nodes_to_visit
        self.__temperature = init_temp
        self.__min_temperature = min_temp
        self.__cool_rate = cool_rate
        self.__max_iters = max_iters

        self.__size_nodes = len(nodes_to_visit)

        self.__current_path = self.__generate_initial_path()
        self.best_path = self.__current_path[:]
        self.best_cost = self.__compute_path_cost(self.best_path)


    def __get_distances_matrix(self):
        """
        Computes the distances matrix for all pairs of nodes in the graph.

        This private method generates a matrix where each element represents the shortest path 
        distance between a pair of nodes in the graph. The distances are computed using Dijkstra's 
        algorithm, assuming the graph is weighted.

        Returns:
            pd.DataFrame: A DataFrame where the rows and columns correspond to the graph's nodes, 
                          and each element [i, j] contains the shortest path distance from node i to node j.

        Notes:
            - The method utilizes NetworkX's shortest_path_length function with a weight parameter 
              to account for edge weights.
            - The resulting DataFrame is symmetric if the graph is undirected, with zeros on the diagonal.
        """
        matrix_distance = dict()

        network_nodes = self.__graph.nodes()
        for origin_node in network_nodes:
            matrix_distance[origin_node] = []
            for target_node in network_nodes:
                distance_by_origin = nx.shortest_path_length(self.__graph, origin_node, target_node, weight="weight")
                matrix_distance[origin_node].append(distance_by_origin)
 
        matrix_distance = pd.DataFrame(matrix_distance)
        matrix_distance.index = network_nodes
        return matrix_distance

    def __generate_initial_path(self):
        """
        Generates the initial path by shuffling the list of nodes to visit.

        Returns:
        list: A shuffled list representing the initial path.
        """
        path = self.__nodes_to_visit[:]
        shuffle(path)
        return path
    
    def __generate_new_path(self, path):
        """
        Generates a new path by swapping two randomly selected nodes in the current path.

        Parameters:
        path (list): The current path.

        Returns:
        list: A new path with two nodes swapped.
        """
        new_path = path[:]
        first_node, second_node = sample(range(self.__size_nodes), 2)
        new_path[first_node], new_path[second_node] = new_path[second_node], new_path[first_node]
        return new_path
    
    def __compute_path_cost(self, path):
        """
        Computes the total cost of the given path based on the distance matrix.

        Parameters:
        path (list): The path for which the cost is to be computed.

        Returns:
        float: The total cost of the path.
        """
        total_cost = 0.0
        for k in range(len(path) - 1):
            total_cost += self.__dist_matrix.loc[path[k], path[k+1]]
        total_cost += self.__dist_matrix.loc[path[-1], path[0]]
        return total_cost

    def __acceptance_condition(self, new_cost, old_cost, temp):
        """
        Determines if the new path should be accepted based on the cost difference and the current temperature.

        Parameters:
        new_cost (float): The cost of the new path.
        old_cost (float): The cost of the current path.
        temp (float): The current temperature.

        Returns:
        float: The acceptance probability.
        """
        if new_cost < old_cost:
            return 1.0
        return exp((old_cost - new_cost) / temp)

    def __get_full_tsp_path(self):
        complete_best_route = []

        for i in range(self.__size_nodes - 1):
            complete_best_route += nx.shortest_path(self.__graph, self.best_path[i], self.best_path[i+1], weight="weight")
        complete_best_route += nx.shortest_path(self.__graph, self.best_path[-1], self.best_path[0], weight="weight")

        self.best_path = complete_best_route
        self.best_cost = self.__compute_path_cost(self.best_path) 
    
    def find_solution(self):
        """
        Executes the simulated annealing algorithm to find the best solution for the TSP.

        Returns:
        tuple: A tuple containing the best path and the best cost.
        """
        actual_cost = self.__compute_path_cost(self.__current_path)

        for iteration in range(self.__max_iters):
            if self.__temperature >= self.__min_temperature:
                new_path = self.__generate_new_path(self.__current_path)
                new_cost = self.__compute_path_cost(new_path)

                if self.__acceptance_condition(new_cost, actual_cost, self.__temperature):
                    self.__current_path = new_path[:]
                    actual_cost = new_cost

                    if new_cost < self.best_cost:
                        self.best_path = self.__current_path[:]
                        self.best_cost = actual_cost

                self.__temperature *= self.__cool_rate
            else:
                break

        self.__get_full_tsp_path()

        print(f"Finished simulated annealing with...\n{iteration+1} iterations and temperature of {self.__temperature}\nBest Solution: {self.best_path}\nBest Cost: {self.best_cost}")

        return self.best_path, self.best_cost


Planteamos el problema de visitar las estaciones UNIVERSIDAD, CONSULADO, PANTEONES, TACUBAYA, CIUDADAZTECA mediante el Recocido Simulado implementado

In [9]:
nodes_to_visit = ["UNIVERSIDAD", "CONSULADO", "PANTEONES", "TACUBAYA", "CIUDADAZTECA"]

sa_tsp = TSP_SimulatedAnnealing(metro_graph, nodes_to_visit, 10.0, 1e-3, 0.99, 50)
best_solution, best_cost = sa_tsp.find_solution()

Finished simulated annealing with...
50 iterations and temperature of 6.050060671375365
Best Solution: ['CIUDADAZTECA', 'PLAZAARAGON', 'OLIMPICA', 'ECATEPEC', 'MUZQUIZ', 'RIOREMEDIOS', 'IMPULSORA', 'NEZAHUALCO', 'VILLAARAGON', 'BOSQUEARAGON', 'DPVOOCEANIA', 'OCEANIA', 'ARAGON', 'EDUARDOMOLI', 'CONSULADO', 'CONSULADO', 'CANALNTE', 'MORELOS', 'TEPITO', 'LAGUNILLA', 'GARIBALDI', 'BELLASARTES', 'HIDALGO', 'REVOLUCION', 'SANCOSME', 'NORMAL', 'COLMILITAR', 'POPOTLA', 'CUITLAHUAC', 'TACUBA', 'PANTEONES', 'PANTEONES', 'TACUBA', 'SANJOAQUIN', 'POLANCO', 'AUDITORIO', 'CONSTITUYENTES', 'TACUBAYA', 'TACUBAYA', 'SANPEDROPINOS', 'SANANTONIO', 'MIXCOAC', 'INSURGENTESSUR', 'HOSPITAL20', 'ZAPATA', 'COYOACAN', 'VIVEROS', 'MAQ', 'COPILCO', 'UNIVERSIDAD', 'UNIVERSIDAD', 'COPILCO', 'MAQ', 'VIVEROS', 'COYOACAN', 'ZAPATA', 'DIVISIONNTE', 'EUGENIA', 'ETIOPIA', 'CENTROMEDICO', 'LAZAROCAR', 'CHABACANO', 'JAMAICA', 'FRAYSERVANDO', 'CANDELARIA', 'SANLAZARO', 'RFLORESMAGON', 'ROMERORUBIO', 'OCEANIA', 'DPVOOCEANIA'

Generamos una clase para hallar mapeos de color para denotar el orden de las estaciones visitadas

In [10]:
class ColorizerByMap:
  def __init__(self, cmap_name, start_val, stop_val):
    self.cmap_name = cmap_name
    self.cmap = plt.get_cmap(cmap_name)
    self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
    self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

  def get_rgb(self, val):
    return self.scalarMap.to_rgba(val, bytes=False, norm=True)

In [11]:
size_tsp_path_SA = len(best_solution)

colorize_tsp_path_SA = ColorizerByMap("coolwarm", 0, size_tsp_path_SA)

colors_tsp_path_greedy = {node:'rgba' + str(colorize_tsp_path_SA.get_rgb(i)) for i, node in enumerate(best_solution)}

Visualizamos el grafo original del sistema de metro de la Ciudad de México con la ruta solución

In [12]:
NodesNames = list(metro_graph.nodes())
colors_nodes = len(NodesNames)*['rgba(0,0,0,1)']
for node in best_solution:
    colors_nodes[NodesNames.index(node)] = colors_tsp_path_greedy[node]

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color='gray'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    textposition="top center",
    marker = dict(
        color = colors_nodes,
        size = 10,
        line = dict(
            color = "black",
            width = 1
        )        
    ),
    line_width=2)

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Ruta obtenida por Greedy',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper")],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)))

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

fig.show()